In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_lattice as tfl
%pylab inline

In [ ]:
# Import helpers
import import_ipynb
from common import buildDatasetForLattice
from common import scaleVolume
from common import extractLatticeWeights
from common import dropColumns
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractXGWeights

In [ ]:
dataset, columns = buildDatasetForLattice()
dataset = filterBad(dataset)

In [ ]:
preprocessed_dataset = dataset.copy()
col = preprocessed_dataset.first_val
preprocessed_dataset.first_val = np.log(col + 1)

In [ ]:
kp_initializers = {
    'row': "quantile", 'rising_idx': "quantile", 'falling_idx': "quantile", 'first_val': "uniform", 'last_val': "uniform"
}
monotonicities = {
    "rising_idx": -1,
    "falling_idx": -1
}

In [ ]:
def buildLattice(num_keypoints, lattice_size, columns, num_lattices):
    inputs = [keras.layers.Input(shape=[1]) for _ in columns]
    combined_calibrators = []
    for inpt, ft in zip(inputs, columns):
        if ft != "row":
            if kp_initializers[ft] == "quantile":
                quantile_vals = [i/(num_keypoints - 1.0) for i in range(num_keypoints)]
                keypoints = dataset[ft].quantile(quantile_vals).values
            if kp_initializers[ft] == "uniform":
                keypoints = np.linspace(preprocessed_dataset[ft].min(), preprocessed_dataset[ft].max(), num=num_keypoints)

            calibrator = tfl.layers.PWLCalibration(
                input_keypoints=keypoints, dtype=tf.float32, output_min=0.0, output_max=lattice_size - 1.0,
                kernel_regularizer = [("wrinkle", 1e-4, 1e-5)],
                monotonicity=monotonicities.get(ft, 0),
                units = num_lattices
            )(inpt)
        else:
            # row is categorical
            calibrator = tfl.layers.CategoricalCalibration(
                num_buckets=preprocessed_dataset[ft].nunique(),
                output_min = 0.0,
                output_max = lattice_size - 1.0,
                units = num_lattices
            )(inpt)
        combined_calibrators.append(calibrator)
    lattices = []
    for i in range(num_lattices):
        lattice_inputs = []
        for col in range(len(columns)):
            lattice_inputs.append(
                keras.backend.transpose(
                    keras.backend.gather(
                        keras.backend.transpose(combined_calibrators[col]), [i])))
        lattice = tfl.layers.Lattice(
            lattice_sizes=[lattice_size for _ in columns],
            monotonicities=['increasing' if (ft == 'rising_idx' or ft == 'falling_idx') else 'none' for x in columns],
            output_min=dataset['delay'].min(),
            output_max=dataset['delay'].max())(keras.layers.concatenate(lattice_inputs))
        lattices.append(lattice)
    
    if num_lattices == 1:
        model = keras.models.Model(inputs=inputs, outputs=lattices[0])
    else:
        model = keras.models.Model(inputs=inputs, outputs=tfl.layers.Linear(
            num_input_dims=num_lattices, normalization_order=1,
            kernel_regularizer=keras.regularizers.l1(0.01))(keras.layers.concatenate(lattices)))
    model.compile(loss=keras.losses.mean_absolute_error,
                optimizer=keras.optimizers.Adam(), metrics=["mse"])
    return model


In [ ]:
def trainLattice(model, cols, epochs):
    features = [preprocessed_dataset[col].values for col in cols]
    target = preprocessed_dataset["delay"]

    model.fit(features,
            target,
            batch_size=32,
            epochs=epochs,
            validation_split=0.2,
            shuffle=True, workers=32, use_multiprocessing=True)
    return model